# Unsupervised learning

Overview of today's topics:
  - linear discriminant analysis
  - principal component analysis
  - k-means clustering
  - DBSCAN clustering
  - hierarchical clustering
  - t-sne projection
  
In unsupervised learning, we use an algorithm to discover structure in and extract information from data. It generally comprises two broad categories:
  - **dimensionality reduction**: transform features to a lower-dimension space
  - **clustering**: assign observations to groups based on their features
  
While supervised learning trains a model to make predictions based on a training data set that we feed it, unsupervised learning discovers relationships and groups automatically for us.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

np.random.seed(0)

In [ ]:
# load CA tract-level census variables
df = pd.read_csv('../../data/census_tracts_data_ca.csv', dtype={'GEOID10':str}).set_index('GEOID10')
df.shape

In [ ]:
df.head()

## 1. Linear discriminant analysis

Dimensionality reduction lets us reduce the number of features (variables) in our data set with minimal loss of information. This data compression is called **feature extraction**. Feature extraction is similar to feature selection in that they both reduce the total number of variables in your analysis. In feature selection, we use domain theory or an algorithm to select important variables for our model. Feature extraction instead projects your features onto a lower-dimension space, creating new features rather than just selecting a subset of existing ones.

LDA is *supervised* dimensionality reduction, providing a link between supervised learning and dimensionality reduction. It uses a categorical response and continuous features to identify features that account for the most variance between classes (ie, maximum separability). It can be used as a classifier, similar to what we saw last week, or it can be used for dimensionality reduction by projecting the features in the most discriminative directions.

We will predict which county a tract is in using 1) a full set of features, and 2) a set of just two projected features. Let's see how it performs.

In [ ]:
# choose response and predictors
response = 'county_name'
features = ['median_age', 'pct_hispanic', 'pct_white', 'pct_black', 'pct_asian', 'pct_male',
            'pct_single_family_home', 'med_home_value', 'med_rooms_per_home', 'pct_built_before_1940',
            'pct_renting', 'rental_vacancy_rate', 'avg_renter_household_size', 'med_household_income',
            'mean_commute_time', 'pct_commute_drive_alone', 'pct_below_poverty', 'pct_college_grad_student',
            'pct_same_residence_year_ago', 'pct_bachelors_degree', 'pct_english_only', 'pct_foreign_born']

In [ ]:
counties = ['Los Angeles', 'Orange', 'Riverside']
mask = df['county_name'].isin(counties)
subset = features + [response]
data = df.loc[mask].dropna(subset=subset)
y = data[response]
X = data[features]
y.shape, X.shape

In [ ]:
# feature scaling
X = StandardScaler().fit_transform(X)

In [ ]:
# reduce data from n dimensions to 2
lda = LinearDiscriminantAnalysis(n_components=2)
X_reduced = lda.fit_transform(X, y)
X_reduced.shape

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
for county_name in data['county_name'].unique():
    mask = y == county_name
    ax.scatter(x=X_reduced[mask, 0],
               y=X_reduced[mask, 1],
               alpha=0.5,
               s=3,
               label=county_name)
ax.set_aspect('equal')
ax.legend(loc='best', scatterpoints=4)
_ = ax.set_title('')

How good are my predictions with just two dimensions? This is a quick and dirty measure of predictive quality between the original, full feature space and the reduced feature space (for a formal analysis, I'd do a test-train split like we saw last week):

In [ ]:
# how accurate are my predictions using all 22 features?
y_pred = LogisticRegression(max_iter=200).fit(X, y).predict(X)
print(round(accuracy_score(y, y_pred), 3))

# how accurate are my predictions using just 2 projected features?
y_pred = LogisticRegression().fit(X_reduced, y).predict(X_reduced)
print(round(accuracy_score(y, y_pred), 3))

We have summarized the most relevant information of our feature space and reduced it from 22 features (ie, dimensions) to just 2. It's not perfect: there has been some information loss, but it's pretty good!

In [ ]:
# now it's your turn
# try changing the number of counties we retain and the number of dimensions
# how does this influence our classification predictions?

## 2. Principal component analysis

Remember simple pair-plots? They let you inspect pairwise relationships between your variables. But what if you have lots of features? PCA offers a more rigorous tool. PCA is very similar to exploratory factor analysis, and is often referred to as a type of factor analysis. The former is used to discover relationships in the data, whereas the latter usually implies that you are probing a theoretical (latent) relationship among your variables. We'll focus on PCA today.

PCA is used 1) to fix multicollinearity problems and 2) for dimensionality reduction. In the former, it converts a set of original, correlated features into a new set of orthogonal features, which is useful in regression and cluster analysis. In the latter, it summarizes a set of original, correlated features with a smaller number of features that still explain most of the variance in your data (data compression).

PCA identifies the combinations of features (directions in feature space) that account for the most variance in the dataset. These orthogonal axes of maximum variance are called principal components. A **principal component** is an eigenvector (direction of maximum variance) of the features' covariance matrix, and the corresponding eigenvalue is its magnitude (factor by which it is "stretched"). An eigenvector is the cosine of the angle between a feature and a component. Its corresponding eigenvalue represents the share of variance it accounts for. PCA takes your (standardized) features' covariance matrix, decomposes it into its eigenvectors/eigenvalues, sorts them by eigenvalue magnitude, constructs a projection matrix $W_k$ from the corresponding top $k$ eigenvectors, then transforms the features using the projection matrix to get the new $k$-dimensional feature subspace. Always standardize your data before PCA because it is sensitive to features' scale.

We will reduce our feature set to fewer dimensions.

In [ ]:
# this is unsupervised, so we don't need a response variable, but we will
# define one just so we can build a simple regression model when we're done
response = 'med_gross_rent'
features = ['median_age', 'pct_hispanic', 'pct_white', 'pct_black', 'pct_asian', 'pct_male',
            'pct_single_family_home', 'med_home_value', 'med_rooms_per_home', 'pct_built_before_1940',
            'pct_renting', 'rental_vacancy_rate', 'avg_renter_household_size', 'med_household_income',
            'mean_commute_time', 'pct_commute_drive_alone', 'pct_below_poverty', 'pct_college_grad_student',
            'pct_same_residence_year_ago', 'pct_bachelors_degree', 'pct_english_only', 'pct_foreign_born']

In [ ]:
subset = features + [response]
data = df.dropna(subset=subset)
y = data[response]
X = data[features]
y.shape, X.shape

In [ ]:
# feature scaling
X = StandardScaler().fit_transform(X)

In [ ]:
# project the features onto all principal components
pca = PCA(n_components=None)
X_reduced = pca.fit_transform(X)

In [ ]:
# our features are correlated with each other, but our principal components are not
pd.DataFrame(X_reduced).corr().round(2)

In [ ]:
# eigenvalues represent the variance explained by each component
# calculate each component's proportion of variance explained
eigenvalues = pca.explained_variance_
pve = eigenvalues / eigenvalues.sum()
pve

In [ ]:
# create a variance-explained plot
xpos = range(1, len(features) + 1)
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(xpos, pve, marker='o', markersize=5, label='Individual')
ax.plot(xpos, np.cumsum(pve), marker='o', markersize=5, label='Cumulative')
ax.set_ylabel('Proportion of variance explained')
ax.set_xlabel('Principal component')
ax.set_xlim(0, len(features) + 1)
ax.set_ylim(0, 1)
ax.grid(True, ls='--')
_ = ax.legend()

So, how many components should we use? Remember, the goal here is to reduce the dimensionality of the feature set: we want to balance parsimony with explanatory power. There is no single answer, but in general you want the fewest components that explain sufficient variation. So what's the right balance?

  - variance-explained criteria: for example, take fewest components necessary to explain, say, 80% of your variance
  - visualization criteria: consider that it is impossible to visualize more than 3 dimensions
  - elbow criteria: use a scree plot (aka, variance-explained plot) and look for an "elbow" in the curve
  - kaiser criteria: use components with an eigenvalue >1 (an obsolete method today)
  
For visualization purposes, let's use two components:

In [ ]:
# project the features onto a 2-dimensional subspace
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
# see our projected features
X_reduced

We often refer to these projected data as "principal component scores" or a "score matrix", $T_k$, where $T_k = XW_k$ and $X$ is your original feature matrix and $W_k$ is the projection matrix, that is, a matrix containing the first $k$ principal components (ie, the $k$ eigenvectors with the largest corresponding eigenvalues). In our case, $k=2$. We can calculate this manually:

In [ ]:
# project our features manually onto the two dimensions
eigenvectors = pca.components_.T
np.dot(X, eigenvectors)

Loadings represent the correlations between the features and the components. Loadings are the eigenvectors scaled by the square roots of their eigenvalues (aka, "singular values").

In [ ]:
eigenvalues = pca.explained_variance_
loadings = eigenvectors * np.sqrt(eigenvalues)

# turn into a DataFrame with column names and row labels
cols = [f'PC{i}' for i in range(1, pca.n_components_ + 1)]
pd.DataFrame(loadings, index=features, columns=cols).sort_values('PC1')

In [ ]:
# how accurate are my predictions using all 22 features?
y_pred = LinearRegression().fit(X, y).predict(X)
print(round(r2_score(y, y_pred), 3))

# how accurate are my predictions using just the first 2 principal components?
y_pred = LinearRegression().fit(X_reduced, y).predict(X_reduced)
print(round(r2_score(y, y_pred), 3))

In [ ]:
# plot the points on their first 2 PCs, and color by the response variable
fig, ax = plt.subplots(figsize=(6, 6))
ax = sns.scatterplot(ax=ax, x=X_reduced[:, 0], y=X_reduced[:, 1],
                     hue=y, palette='plasma_r', s=5, edgecolor='none')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
_ = ax.set_aspect('equal')

In [ ]:
# now it's your turn
# conduct a PCA with the first 4 components
# how does our predictive accuracy change? how does the scatterplot change?

## 3. k-means clustering

Dimensionality reduction projects our data onto a lower-dimension space, usually through unsupervised learning. A second branch of unsupervised learning, **cluster analysis**, lets us discover natural groups that exist in our data. Last week we predicted groups in labeled data by training a supervised learning algorithm. In cluster analysis, we discover unknown groups in unlabeled data through an unsupervised learning algorithm. As with dimensionality reduction, remember to standardize your data before clustering. Many clustering algorithms work well in high-dimensional feature spaces, but some work better after PCA dimensionality reduction (due to the curse of dimensionality).

**k-means** is probably the most common clustering algorithm. It clusters data into $k$ groups based on their similarity. It is a form of **prototype-based clustering** where each cluster is represented by a prototype, or centroid. You have to specify the number of groups in advance. This works well when you want to partition your data into a predetermined number of groups. Otherwise, you have to determine an optimal value for $k$.

Here, we will identify counties that are similar to one another across a wide variety of characteristics.

In [ ]:
features = ['median_age', 'pct_hispanic', 'pct_white', 'pct_black', 'pct_asian', 'pct_male', 'med_gross_rent',
            'pct_single_family_home', 'med_home_value', 'med_rooms_per_home', 'pct_built_before_1940',
            'pct_renting', 'rental_vacancy_rate', 'avg_renter_household_size', 'med_household_income',
            'mean_commute_time', 'pct_commute_drive_alone', 'pct_below_poverty', 'pct_college_grad_student',
            'pct_same_residence_year_ago', 'pct_bachelors_degree', 'pct_english_only', 'pct_foreign_born']

In [ ]:
# calculate then standardize median values across counties
counties = df.groupby('county_name').median()
X = counties[features].dropna()
X = StandardScaler().fit_transform(X)
X.shape

In [ ]:
# project onto first two principal components for 2-D clustering
X_reduced = PCA(n_components=2).fit_transform(X)
X_reduced.shape

In [ ]:
# cluster the data
km = KMeans(n_clusters=5).fit(X_reduced)

In [ ]:
# get the cluster labels, the unique labels, and the number of clusters obtained
cluster_labels = km.labels_
unique_labels = set(cluster_labels)
num_clusters = len(unique_labels)
print(f'Number of clusters: {num_clusters}')

In [ ]:
pd.Series(cluster_labels).value_counts().sort_index()

In [ ]:
# scatterplot points on first two PCs and color by cluster
fig, ax = plt.subplots(figsize=(4, 4))
ax = sns.scatterplot(ax=ax, x=X_reduced[:, 0], y=X_reduced[:, 1],
                     hue=cluster_labels, palette='Set1', s=20, edgecolor='none')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
_ = ax.set_aspect('equal')

In [ ]:
# silhouette score is the average silhouette coefficient
silhouette_score(X_reduced, cluster_labels)

The **silhouette** score measures *cohesion* vs *separation*: how similar the points are to to their own clusters vs to the other clusters, on average. This measures how tightly grouped our clusters are. The silhouette can range from -1 to +1. Negative values suggest clustering problems, including too many/few clusters.

So how do you pick a good $k$?
  - theoretically, how many clusters should there be in your data (if knowable beforehand)?
  - which $k$ value gives you the best silhouette score?
  - elbow criteria (similar to what we saw for PCA): find an elbow in the line plot of distortion vs cluster count. Distortion is also called inertia, and represents the sum of squared errors.

In [ ]:
# create an elbow plot
fig, ax = plt.subplots()
ax.set_xlabel('Number of clusters')
ax.set_ylabel('Distortion')
kvals = range(1, 15)
distortions = []
for k in kvals:
    km = KMeans(n_clusters=k).fit(X_reduced)
    distortions.append(km.inertia_)
ax.plot(kvals, distortions, marker='o')
_ = ax.grid(True)

In [ ]:
# now it's your turn
# use the elbow plot above to choose a new k value
# how does it affect the scatterplot and silhouette score?

## 4. DBSCAN clustering

DBSCAN (density-based spatial clustering of applications with noise) represents another form of clustering known as **density-based clustering**. Density-based clustering works better in low-dimension feature spaces, so PCA in advance is a good idea.

DBSCAN assigns cluster labels based on dense regions of points, by identifying core points, border points, and noise points. Unlike k-means, we do not need to know the number of clusters beforehand. We parameterize it with a minimum number of points that must fall within some radius $\epsilon$ of a point to consider that point a core point. The $\epsilon$ parameter represents the maximum distance in the feature space that points can be from each other to be considered a cluster. The min_samples parameter is the minimum cluster size allowed: everything else gets classified as noise.

DBSCAN can be useful for geospatial clustering of either projected coordinates, or lat-long coordinates if you use a haversine distance metric. But here, we will just cluster our same features as before.

In [ ]:
# cluster the data (in two dimensions again)
X_reduced = PCA(n_components=2).fit_transform(X)
db = DBSCAN(eps=1, min_samples=3, metric='euclidean').fit(X_reduced)

In [ ]:
# get the cluster labels, the unique labels, and the number of clusters obtained
cluster_labels = db.labels_
unique_labels = set(cluster_labels)
num_clusters = len(unique_labels)
print(f'Number of clusters: {num_clusters}')

In [ ]:
# scatterplot points on first two PCs and color by cluster
# cluster label -1 means noise
fig, ax = plt.subplots(figsize=(4, 4))
ax = sns.scatterplot(ax=ax, x=X_reduced[:, 0], y=X_reduced[:, 1],
                     hue=cluster_labels, palette='Set1', s=20, edgecolor='none')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
_ = ax.set_aspect('equal')

In [ ]:
silhouette_score(X_reduced, cluster_labels)

In [ ]:
# now it's your turn
# try changing the epsilon and min_samples then re-clustering
# how does it change the silhouette score and the cluster plot?

## 5. Hierarchical clustering

Another form of clustering is **hierarchical clustering**, which can be agglomerative or divisive. Agglomerative clustering initially treats each observation as its own cluster, then iteratively merges the closest two clusters until only one supercluster remains. There are four common algorithms:
  - single linkage: calculate distance between the most similar members in each pair of clusters, then merge the two clusters with smallest such distance
  - complete linkage: like single linkage, but instead compare the most dissimilar members
  - average linkage: calculate average distance between all members in each pair of clusters, then merge the two clusters with smallest average distance
  - Ward's linkage: merge the two clusters that cause the least increase in total within-cluster sum of squared errors
  
We could use scikit-learn, but I prefer agglomerative clustering in scipy so we can easily visualize the dendrogram. A dendrogram shows us how the clusters link up and lets us explore which observations are more/less similar. The dendrogram's structure suggests high-level superclusters and we can cut its tree at an arbitrary level.

In this example, we'll cluster in four dimensions, which was suggested by our PCA variance-explained plot earlier.

In [ ]:
# project onto first 4 principal components
X_reduced = PCA(n_components=4).fit_transform(X)
X_reduced.shape

In [ ]:
# calculate distance matrix then linkage matrix, choosing a method (algorithm)
distances = pdist(X_reduced)
Z = hierarchy.linkage(distances, method='complete', optimal_ordering=True)

In [ ]:
# cophenetic correlation measures how well clustering preserved pairwise distances
c, _ = hierarchy.cophenet(Z, distances)
c

In [ ]:
# pick a distance to cut dendrogram tree
cut_point = 6

# plot the dendrogram, colored by clusters below the cut point
fig, ax = plt.subplots(figsize=(5, 11))
ax.set_xlabel('Euclidean distance')
with plt.rc_context({'lines.linewidth': 1}):
    R = hierarchy.dendrogram(Z=Z,
                             orientation='right',
                             labels=counties.index,
                             color_threshold=cut_point,
                             distance_sort='descending',
                             show_leaf_counts=False,
                             ax=ax)
plt.axvline(cut_point, c='k')
fig.savefig('dendrogram.png', dpi=600, facecolor='w', bbox_inches='tight')

In [ ]:
# assign k cluster labels to the observations, based on where you cut tree
# k = number of clusters = how many horizontal lines you intersected above
k = 8
cluster_labels = hierarchy.fcluster(Z, t=k, criterion='maxclust')
pd.Series(cluster_labels).value_counts().sort_index()

In [ ]:
# now it's your turn
# pick different points to cut the tree, how many clusters do they imply?
# which cut point is the right one to use?

## 6. t-SNE

What if I want to discover structure in >3 dimensions (like we did above), but still be able to visualize it?

Manifold learning is a nonlinear dimensionality reduction approach that usually uses unsupervised learning. **t-SNE** (t-distributed stochastic neighbor embedding) is a manifold learning technique used for projecting high-dimension data sets into a plane for easy visualization. Here, we project our counties' higher-dimension feature space to 2 dimensions for visualization. t-SNE projection is useful because it preserves group structure relatively well despite information loss. However, given the global density-equalizing nature of t-SNE, relative distances within and between clusters are not preserved and should not be interpreted otherwise. 

For an example of using clustering + t-SNE to discover and visualize similar places, see this [ANS article](https://doi.org/10.1007/s41109-019-0189-1). Here, we will use t-SNE to project our data to two dimensions to scatterplot the hierarchical clusters from above.

In [ ]:
# t-SNE with two dimensions, then project features onto this space
tsne = TSNE(n_components=2, n_iter=10000, random_state=0)
X_reduced = pd.DataFrame(data=tsne.fit_transform(X),
                         index=counties.index,
                         columns=['TC1', 'TC2'])

In [ ]:
# plot the colored clusters projected onto the two t-SNE dimensions
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
X_reduced['color'] = pd.Series(dict(zip(R['ivl'], R['leaves_color_list'])))
ax.scatter(x=X_reduced['TC1'], y=X_reduced['TC2'], c=X_reduced['color'], s=10)

# identify a county of interest in the plot
county = 'San Francisco'
_ = ax.scatter(x=X_reduced.loc[county, 'TC1'],
               y=X_reduced.loc[county, 'TC2'],
               alpha=1, marker='o', s=300, linewidth=2, color='none', ec='k')

In [ ]:
# now it's your turn
# pick different points to cut the tree, how does it change our t-SNE plot?